In [64]:
import pandas as pd
import sqlite3

In [65]:
#read information data for dataframe enrichement (merges)
airports = pd.read_csv('airports.csv')
carriers = pd.read_csv('carriers.csv')
plane_data = pd.read_csv('plane-data.csv')

In [66]:
airports

,iata,airport,city,state,country,lat,long
0,00M,Thigpen,Bay Springs,MS,USA,31.953765,-89.234505
1,00R,Livingston Municipal,Livingston,TX,USA,30.685861,-95.017928
2,00V,Meadow Lake,Colorado Springs,CO,USA,38.945749,-104.569893
3,01G,Perry-Warsaw,Perry,NY,USA,42.741347,-78.052081
4,01J,Hilliard Airpark,Hilliard,FL,USA,30.688012,-81.905944
...,...,...,...,...,...,...,...
3371,ZEF,Elkin Municipal,Elkin,NC,USA,36.280024,-80.786069
3372,ZER,Schuylkill Cty/Joe Zerbey,Pottsville,PA,USA,40.706449,-76.373147
3373,ZPH,Zephyrhills Municipal,Zephyrhills,FL,USA,28.228065,-82.155916
3374,ZUN,Black Rock,Zuni,NM,USA,35.083227,-108.791777


In [67]:
#cancellation reason dictionary from variable description for dataframe enrichement
cancellation_reasons = {'A': 'carrier', 'B': 'weather', 'C': 'NAS', 'D': 'security'}

In [68]:
df_2004 = pd.read_csv('2004.csv.bz2', compression='bz2')
df_2005 = pd.read_csv('2005.csv.bz2', compression='bz2')
df_2006 = pd.read_csv('2006.csv.bz2', compression='bz2')
df_2007 = pd.read_csv('2007.csv.bz2', compression='bz2')
df_2008 = pd.read_csv('2008.csv.bz2', compression='bz2')

In [69]:
# Convert dictionary to DataFrame
df_cancellation_reasons = pd.DataFrame(
    cancellation_reasons.items(),
    columns=['id', 'reason']
)


In [70]:
dataframes = {
    'airports': airports,
    'carriers': carriers,
    'plane_data': plane_data,
    'cancellation_reason': df_cancellation_reasons,
    '2004': df_2004,
    '2005': df_2005,
    '2006': df_2006,
    '2007': df_2007,
    '2008': df_2008
}


In [71]:
db_path = 'flight_data.db'

# Open connection
with sqlite3.connect(db_path) as conn:
    for table_name, df in dataframes.items():
        df.to_sql(table_name, conn, if_exists='replace', index=False)

In [40]:
df_list = [df_2004, df_2005, df_2006, df_2007, df_2008]
airports_origin = airports.rename(
    columns=lambda col: col + '_origin' if col != 'iata' else col
)
df_list = [
    pd.merge(df, airports_origin, left_on='Origin', right_on='iata', how='left')
    for df in df_list
]

In [44]:
airports_destination = airports.rename(
    columns=lambda col: col + '_destination' if col != 'iata' else col
)
df_list = [
    pd.merge(df, airports_destination, left_on='Dest', right_on='iata', how='left')
    for df in df_list
]

In [50]:
df_list = [
    pd.merge(df, carriers, left_on='UniqueCarrier', right_on='Code', how='left')
    for df in df_list
]

In [51]:
df_list[0]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,long_origin,iata_y,airport_destination,city_destination,state_destination,country_destination,lat_destination,long_destination,Code,Description
0,2004,1,12,1,623.0,630,901.0,915,UA,462,...,-87.904464,CLT,Charlotte/Douglas International,Charlotte,NC,USA,35.214011,-80.943126,UA,United Air Lines Inc.
1,2004,1,13,2,621.0,630,911.0,915,UA,462,...,-87.904464,CLT,Charlotte/Douglas International,Charlotte,NC,USA,35.214011,-80.943126,UA,United Air Lines Inc.
2,2004,1,14,3,633.0,630,920.0,915,UA,462,...,-87.904464,CLT,Charlotte/Douglas International,Charlotte,NC,USA,35.214011,-80.943126,UA,United Air Lines Inc.
3,2004,1,15,4,627.0,630,859.0,915,UA,462,...,-87.904464,CLT,Charlotte/Douglas International,Charlotte,NC,USA,35.214011,-80.943126,UA,United Air Lines Inc.
4,2004,1,16,5,635.0,630,918.0,915,UA,462,...,-87.904464,CLT,Charlotte/Douglas International,Charlotte,NC,USA,35.214011,-80.943126,UA,United Air Lines Inc.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7129265,2004,12,16,4,1645.0,1639,1715.0,1716,DL,1138,...,-84.426944,MSY,New Orleans International,New Orleans,LA,USA,29.993389,-90.258028,DL,Delta Air Lines Inc.
7129266,2004,12,16,4,726.0,720,1238.0,1239,DL,1139,...,-110.941028,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944,DL,Delta Air Lines Inc.
7129267,2004,12,16,4,1336.0,1330,1507.0,1504,DL,1139,...,-84.426944,DCA,Ronald Reagan Washington National,Arlington,VA,USA,38.852083,-77.037722,DL,Delta Air Lines Inc.
7129268,2004,12,16,4,538.0,540,720.0,723,DL,1140,...,-80.095594,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944,DL,Delta Air Lines Inc.
